In [2]:
from selenium import webdriver
import os
import requests
import pickle
import numpy as np
import pandas as pd

In [27]:
import re

In [3]:
product_url = r'https://item.jd.com/5561746.html#none'

In [4]:
driver = webdriver.Chrome()

In [5]:
driver.get(product_url)

In [94]:
good_comments_url = r'https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId=5561746&score=3&sortType=5&page=0&pageSize=10&isShadowSku=0&fold=1'
bad_comments_url = r'https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId=5561746&score=1&sortType=5&page=0&pageSize=10&isShadowSku=0&fold=1'

In [182]:
header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
}
param = {
    'callback': 'fetchJSON_comment98',
    'productId': '5561746',
    'score': '3',
    'sortType': '5',
    'page': '',
    'pageSize': '10',
    'isShadowSku': '0',
    'fold': '1',
}
cookie = {
    'Cookie': 'shshshfpa=cb4386b6-c0f2-68a1-b156-2236f499ee30-1590065631; shshshfpb=dXHF9pqH0l8XV8dgbxTlNEQ%3D%3D; __jdu=15900656294791955369661; user-key=3e85b9e4-c7cf-43fd-9e1a-756e9776cab0; cn=0; __jdc=122270672; areaId=19; ipLoc-djd=19-1607-3155-0; shshshfp=83f76a7577a1d3cdcb7f20cd9a99ba87; __jdv=122270672|github.com|-|referral|-|1617588163257; jwotest_product=99; __jda=122270672.15900656294791955369661.1590065629.1617593724.1617596142.26; shshshsID=273af6177249142f26677cc915a91991_2_1617597325824; __jdb=122270672.2.15900656294791955369661|26.1617596142; JSESSIONID=EDBD2A6AD8E18761AFF846595478748F.s1; 3AB9D23F7A4B3C9B=NYA7Y2IYQW7V35YN3PSDHABICJZ5GIKPEEIE6XO7TSEUVYNHVZ7CFQHTY2RYGTGNNEFG2YNVNV5ZYJC36L2IOMHRSM'
}

In [153]:
session = requests.Session()
cookie = requests.utils.cookiejar_from_dict(cookie)
session.cookies = cookie
res = session.get(url=good_comments_url, headers=header, data=param)

In [154]:
good_comments = re.findall(r'fetchJSON_comment98\((.*)\)', res.text)[0]

In [155]:
type(good_comments)

str

In [103]:
import json

In [104]:
good_comments = json.loads(good_comments)

In [105]:
type(good_comments)

dict

In [148]:
from tqdm import tqdm
import time

In [183]:
def get_good_comments(header, param, cookie, scrap_page=200):
    session = requests.Session()
    session.cookies = requests.utils.cookiejar_from_dict(cookie)
    scrap_page = scrap_page
    fail = 0
    comments = []
    for page in tqdm(range(scrap_page)):
        good_comments_url = f'https://club.jd.com/comment/productPageComments.action?callback=fetchJSON_comment98&productId=5561746&score=3&sortType=5&page={page}&pageSize=10&isShadowSku=0&fold=1'
        try:
            res = session.get(url=good_comments_url, headers=header, data=param)
            good_comments = re.findall(r'fetchJSON_comment98\((.*)\)', res.text)[0] # 本来获取的是json数据，但是前面加了字符串，所以要删除
            good_comments = json.loads(good_comments) # 将字符串转换成字典
            for itm in good_comments['comments']:
                comments.append(itm['content'])
            time.sleep(1)
        except Exception as e:
            fail += 1
            continue
    print(f'\n成功的页数为:{scrap_page - fail}失败的页数为:{fail}')
    return comments

In [184]:
good_comments = get_good_comments(header=header, param=param, cookie=cookie, scrap_page=150)

100%|██████████| 300/300 [02:59<00:00,  1.67it/s]
成功的页数为:119失败的页数为:181



In [185]:
len(good_comments)

1000

In [179]:
with open('./data/good_comments.pkl', 'wb') as f:
    pickle.dump(good_comments, f)

In [180]:
t = pickle.load(open('./data/good_comments.pkl', 'rb'))

In [ ]:
def get_good_comments(header, param, cookie, scrap_page=200):
    session = requests.Session()
    session.cookies = requests.utils.cookiejar_from_dict(cookie)
    scrap_page = scrap_page
    fail = 0
    comments = []
    for page in tqdm(range(scrap_page)):
        good_comments_url = f'https://club.jd.com/comment/productPageComments.action'
        try:
            res = session.get(url=good_comments_url, headers=header, data=param)
            good_comments = re.findall(r'fetchJSON_comment98\((.*)\)', res.text)[0] # 本来获取的是json数据，但是前面加了字符串，所以要删除
            good_comments = json.loads(good_comments) # 将字符串转换成字典
            for itm in good_comments['comments']:
                comments.append(itm['content'])
            time.sleep(1)
        except Exception as e:
            fail += 1
            continue
    print(f'\n成功的页数为:{scrap_page - fail}失败的页数为:{fail}')
    return comments